In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/" )
mnist = tf.contrib.learn.datasets.load_dataset('mnist')
import numpy as np

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
timesteps = 28
num_classes = 10
num_input = 28

X = tf.placeholder(tf.float32 , [None , timesteps , num_input])
Y = tf.placeholder(tf.int32 , [None ])

In [4]:
x = tf.unstack(X , timesteps ,1)

In [5]:
y = tf.one_hot(Y , num_classes)

In [6]:
num_hidden = 128 # ht 의 수.
lstm_cell = tf.nn.rnn_cell.LSTMCell(num_hidden , forget_bias=1.0 )

In [7]:
#outputs = ht,states = Ct
outputs,states = tf.nn.static_rnn(lstm_cell , x , dtype=tf.float32)

In [8]:
num_units = 256
W1 = tf.Variable(tf.random_normal([num_hidden , num_units]))
b1 = tf.Variable(tf.random_normal([num_units]))

W2 = tf.Variable(tf.random_normal([num_units , num_classes]))
b2 = tf.Variable(tf.random_normal([num_classes]))

In [10]:
hidden_layer = tf.nn.relu(tf.matmul(outputs[-1] , W1)+b1)
logits = tf.matmul(hidden_layer , W2) +b2
prediction = tf.nn.softmax(logits)

In [11]:
cross_ent = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits , labels = y)
loss_op = tf.reduce_mean(cross_ent)

In [12]:
learning_rate = 0.001
opt = tf.train.GradientDescentOptimizer(learning_rate)
train_op = opt.minimize(loss_op)

In [13]:
corr_pred = tf.equal(tf.argmax(prediction , 1) , tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(corr_pred, tf.float32))

In [14]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [15]:
trn_data = mnist.train.images.reshape((-1 , timesteps , num_input))
trn_label = mnist.train.labels
vld_data = mnist.validation.images.reshape((-1 , timesteps , num_input))
vld_label = mnist.validation.labels

In [16]:
trn_data.shape

(55000, 28, 28)

In [17]:
def print_step(step , loss , acc):
    print('step ' + str(step) + ',minibatch lostt = ' +\
         '{:.4f}'.format(loss) + ', trainging Accuracy=' +\
         '{:.3f}'.format(acc))

In [21]:
batch_size = 128
training_steps = 1000

for step  in range(1,training_steps+1):    
    batch_x , batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size , timesteps, num_input))
    _ = sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
    if step % 100 == 0:
        train_loss , train_acc = sess.run([loss_op , accuracy] , feed_dict={X:trn_data , Y:trn_label})
        vld_loss , vld_acc = sess.run([loss_op , accuracy],
                                     feed_dict={X:vld_data,
                                               Y:vld_label})


#             print_step(step , loss , acc)
        print_step_2(step , vld_loss , vld_acc)
print('fin')

step  100 , Minibatch Loss = 1.0459 , training Accuracy=0.662
step  200 , Minibatch Loss = 0.9274 , training Accuracy=0.699
step  300 , Minibatch Loss = 0.8938 , training Accuracy=0.703
step  400 , Minibatch Loss = 0.8246 , training Accuracy=0.727
step  500 , Minibatch Loss = 0.8416 , training Accuracy=0.719
step  600 , Minibatch Loss = 0.7585 , training Accuracy=0.745
step  700 , Minibatch Loss = 0.7203 , training Accuracy=0.766
step  800 , Minibatch Loss = 0.6613 , training Accuracy=0.787
step  900 , Minibatch Loss = 0.6490 , training Accuracy=0.799
step 1000 , Minibatch Loss = 0.6397 , training Accuracy=0.798
fin


In [20]:
def print_step_2(step , loss , acc):
    print('step {:4d} , Minibatch Loss = {:.4f} , training Accuracy={:.3f}'.format(step,loss,acc))